In [1]:
!pip uninstall -y torch torchvision torchaudio
!pip cache purge  # optional, but helps avoid conflicts
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128


ERROR: Too many arguments


Looking in indexes: https://download.pytorch.org/whl/cu128
  Using cached https://download.pytorch.org/whl/cu128/torch-2.9.1%2Bcu128-cp310-cp310-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cu128/torchvision-0.24.1%2Bcu128-cp310-cp310-win_amd64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu128/torchaudio-2.9.1%2Bcu128-cp310-cp310-win_amd64.whl.metadata (7.0 kB)
   ---------------------------------------- 0.0/2.9 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 GB 7.2 MB/s eta 0:06:37
   ---------------------------------------- 0.0/2.9 GB 7.2 MB/s eta 0:06:38
   ---------------------------------------- 0.0/2.9 GB 8.1 MB/s eta 0:05:52
   ---------------------------------------- 0.0/2.9 GB 8.8 MB/s eta 0:05:27
   ---------------------------------------- 0.0/2.9 GB 9.2 MB/s eta 0:05:10
   ---------------------------------------- 0.0/2.9 GB 8.5 MB/s eta 0:05:36
   ---------------------------------------- 0.0/2.9

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.__version__)
print(torch.version.cuda)


cuda
2.9.1+cu128
12.8


## Load data

In [2]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoImageProcessor, AutoModelForImageClassification

# ---------- 1. Dataset from numpy ----------

class NumpyImageDataset(Dataset):
    """
    Expects:
      X: (N, H, W, 3) numpy array
      y: (N,) numpy array of ints (class ids)
    """
    def __init__(self, x_path, y_path):
        self.images = np.load(x_path)   # HWC
        self.labels = np.load(y_path)   # (N,)

        if self.images.ndim != 4 or self.images.shape[-1] != 3:
            raise ValueError(f"Expected images of shape (N, H, W, 3), got {self.images.shape}")

        if len(self.images) != len(self.labels):
            raise ValueError("X and y must have the same length")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]    # numpy, shape (H, W, 3)
        label = int(self.labels[idx])
        return img, label


def make_dataloader(x_path, y_path, image_processor, batch_size=32, shuffle=False):
    """
    Wraps the dataset and uses Hugging Face image_processor
    in a custom collate_fn to:
      - resize to 224x224 (or whatever the model expects)
      - normalize with the correct mean/std
      - return pixel_values as torch.FloatTensor
    """
    dataset = NumpyImageDataset(x_path, y_path)

    def collate_fn(batch):
        # batch is a list of (img, label)
        imgs, labels = zip(*batch)   # imgs: tuple of np arrays (H, W, 3)

        # Optionally ensure uint8 (HF can also handle float images)
        imgs = [img.astype("uint8") for img in imgs]

        encodings = image_processor(
            imgs,
            return_tensors="pt"
            # by default does resize, center crop (if configured),
            # convert to CHW, normalize with mean/std
        )
        pixel_values = encodings["pixel_values"]     # (B, 3, H', W')
        labels = torch.tensor(labels, dtype=torch.long)

        return pixel_values, labels

    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=collate_fn,
    )
    return loader

c:\Users\Huize\miniconda3\envs\ml-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# from importlib import reload
# import src.load_data
# reload(src.load_data)
# from src.load_data import make_dataloader

data_dir = "dataset"

x_train_path = os.path.join(data_dir, "x_train.npy")
y_train_path = os.path.join(data_dir, "y_train.npy")
x_val_path   = os.path.join(data_dir, "x_val.npy")
y_val_path   = os.path.join(data_dir, "y_val.npy")
x_test_path  = os.path.join(data_dir, "x_test.npy")
y_test_path  = os.path.join(data_dir, "y_test.npy")

model_name = "facebook/deit-tiny-patch16-224"

# 1) Hugging Face preprocessing config
image_processor = AutoImageProcessor.from_pretrained(model_name)

# 2) Dataloaders that use the image_processor internally
train_loader = make_dataloader(
    x_train_path, y_train_path,
    image_processor,
    batch_size=32,
    shuffle=True,
)

val_loader = make_dataloader(
    x_val_path, y_val_path,
    image_processor,
    batch_size=32,
    shuffle=False,
)

test_loader = make_dataloader(
    x_test_path, y_test_path,
    image_processor,
    batch_size=32,
    shuffle=False,
)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


## Load model and performance **w/o** fine-tuning

In [4]:
import torch
from transformers import AutoModelForImageClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_name = "facebook/deit-tiny-patch16-224"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=2,                        # binary classification
    ignore_mismatched_sizes=True,       # re-init head if needed
    id2label={0: "civilian", 1: "military"},
    label2id={"civilian": 0, "military": 1},
).to(device)

model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for pixel_values, labels in test_loader:   # <-- use test_loader here
        pixel_values = pixel_values.to(device)
        labels = labels.to(device)

        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits                # (B, 2)
        preds = torch.argmax(logits, dim=-1)   # (B,)

        all_labels.extend(labels.cpu().tolist())
        all_preds.extend(preds.cpu().tolist())

# Convert to metrics
all_labels = torch.tensor(all_labels)
all_preds = torch.tensor(all_preds)

acc = accuracy_score(all_labels, all_preds)

# Treat "military" (label 1) as the positive class
precision, recall, f1, _ = precision_recall_fscore_support(
    all_labels,
    all_preds,
    average="binary",
    pos_label=1,
)

print(f"Test Accuracy : {acc:.4f}")
print(f"Precision (military=1): {precision:.4f}")
print(f"Recall (military=1)   : {recall:.4f}")
print(f"F1 (military=1)       : {f1:.4f}")


Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/deit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([2, 192]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Test Accuracy : 0.5342
Precision (military=1): 0.7857
Recall (military=1)   : 0.0940
F1 (military=1)       : 0.1679


## fine-tuning

In [5]:
import torch
from torch.optim import AdamW
from transformers import AutoModelForImageClassification

for params in model.vit.parameters():
    params.require_grad = False

# Only optimize trainable params (classifier)
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                  lr=5e-5)



num_epochs = 10  # you can increase this


for ep in range(num_epochs):
    # ---------- TRAIN ----------
    model.train()
    running_train_loss = 0.0
    total_train = 0
    correct_train = 0

    for pixel_values, labels in train_loader:
        pixel_values = pixel_values.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # HF model returns loss when labels provided
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        logits = outputs.logits  # (B, 2)

        loss.backward()
        optimizer.step()

        running_train_loss += loss.item() * labels.size(0)

        # Train accuracy
        preds = torch.argmax(logits, dim=-1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    epoch_train_loss = running_train_loss / total_train
    train_acc = correct_train / total_train

    # ---------- VALIDATION ----------
    model.eval()
    running_val_loss = 0.0
    total_val = 0
    correct_val = 0

    with torch.no_grad():
        for pixel_values, labels in val_loader:
            pixel_values = pixel_values.to(device)
            labels = labels.to(device)

            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            running_val_loss += loss.item() * labels.size(0)

            preds = torch.argmax(logits, dim=-1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)

    epoch_val_loss = running_val_loss / total_val
    val_acc = correct_val / total_val

    print(
        f"Epoch {ep+1}/{num_epochs} | "
        f"Train loss: {epoch_train_loss:.4f}, acc: {train_acc:.4f} | "
        f"Val loss: {epoch_val_loss:.4f}, acc: {val_acc:.4f}"
    )


Epoch 1/10 | Train loss: 0.1811, acc: 0.9357 | Val loss: 0.0723, acc: 0.9786
Epoch 2/10 | Train loss: 0.0171, acc: 0.9957 | Val loss: 0.0368, acc: 0.9893
Epoch 3/10 | Train loss: 0.0052, acc: 1.0000 | Val loss: 0.0665, acc: 0.9850
Epoch 4/10 | Train loss: 0.0074, acc: 0.9982 | Val loss: 0.0741, acc: 0.9807
Epoch 5/10 | Train loss: 0.0014, acc: 0.9994 | Val loss: 0.5359, acc: 0.8822
Epoch 6/10 | Train loss: 0.0167, acc: 0.9963 | Val loss: 0.0413, acc: 0.9893
Epoch 7/10 | Train loss: 0.0110, acc: 0.9945 | Val loss: 0.1081, acc: 0.9700
Epoch 8/10 | Train loss: 0.0182, acc: 0.9927 | Val loss: 0.0635, acc: 0.9829
Epoch 9/10 | Train loss: 0.0012, acc: 1.0000 | Val loss: 0.0466, acc: 0.9850
Epoch 10/10 | Train loss: 0.0009, acc: 1.0000 | Val loss: 0.0378, acc: 0.9893


## evaluate fine-tuned model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)
all_labels = []
all_preds = []

with torch.no_grad():
    for pixel_values, labels in test_loader:   # <-- use test_loader here
        pixel_values = pixel_values.to(device)
        labels = labels.to(device)

        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits                # (B, 2)
        preds = torch.argmax(logits, dim=-1)   # (B,)

        all_labels.extend(labels.cpu().tolist())
        all_preds.extend(preds.cpu().tolist())

# Convert to metrics
all_labels = torch.tensor(all_labels)
all_preds = torch.tensor(all_preds)

acc = accuracy_score(all_labels, all_preds)

# Treat "military" (label 1) as the positive class
precision, recall, f1, _ = precision_recall_fscore_support(
    all_labels,
    all_preds,
    average="binary",
    pos_label=1,
)

print(f"Test Accuracy : {acc:.4f}")
print(f"Precision (military=1): {precision:.4f}")
print(f"Recall (military=1)   : {recall:.4f}")
print(f"F1 (military=1)       : {f1:.4f}")


Test Accuracy : 0.9829
Precision (military=1): 0.9913
Recall (military=1)   : 0.9744
F1 (military=1)       : 0.9828


## save with onnx

In [14]:
!pip install onnx onnxruntime

   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.8/16.5 MB 10.1 MB/s eta 0:00:02
   -------- ------------------------------- 3.7/16.5 MB 10.9 MB/s eta 0:00:02
   ------------- -------------------------- 5.5/16.5 MB 9.6 MB/s eta 0:00:02
   ----------------- ---------------------- 7.3/16.5 MB 9.2 MB/s eta 0:00:01
   ---------------------- ----------------- 9.4/16.5 MB 9.6 MB/s eta 0:00:01
   ---------------------------- ----------- 11.8/16.5 MB 10.0 MB/s eta 0:00:01
   ---------------------------------- ----- 14.2/16.5 MB 10.1 MB/s eta 0:00:01
   -------------------------------------- - 16.0/16.5 MB 10.0 MB/s eta 0:00:01
   ---------------------------------------- 16.5/16.5 MB 9.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/13.5 MB 3.0 MB/s eta 0:00:05
   ---- ----------------------------------- 1.6/13.5 MB 3.8 MB/s eta 0:00:04
   -

In [16]:
!pip install onnxscript

   ---------------------------------------- 0.0/683.0 kB ? eta -:--:--
   --------------- ------------------------ 262.1/683.0 kB ? eta -:--:--
   ---------------------------------------- 683.0/683.0 kB 1.6 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [onnxscript]
   -------------------- ------------------- 1/2 [onnxscript]
   -------------------- ------------------- 1/2 [onnxscript]
   -------------------- ------------------- 1/2 [onnxscript]
   ---------------------------------------- 2/2 [onnxscript]



## separate .data

In [17]:
import torch
import torch.nn as nn

# 1) Make sure your model is loaded and fine-tuned already.
# model = AutoModelForImageClassification.from_pretrained("ships-vit-tiny", ...)

model.eval()

# Optional: export from CPU for fewer surprises
device = torch.device("cpu")
model.to(device)

# 2) Wrap the HF model so forward() returns just logits
class LogitsWrapper(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model

    def forward(self, pixel_values):
        # pixel_values: (B, 3, 224, 224), already preprocessed
        outputs = self.base_model(pixel_values=pixel_values)
        return outputs.logits  # (B, 2)

wrapped_model = LogitsWrapper(model).to(device)
wrapped_model.eval()

# 3) Dummy input in the same format as you use at inference
dummy_input = torch.randn(1, 3, 224, 224, device=device)

# 4) Export to ONNX
onnx_path = "ships_vit_tiny.onnx"

torch.onnx.export(
    wrapped_model,
    dummy_input,                  # model input (positional for pixel_values)
    onnx_path,
    input_names=["pixel_values"],
    output_names=["logits"],
    dynamic_axes={
        "pixel_values": {0: "batch_size"},
        "logits": {0: "batch_size"},
    },
    opset_version=17,             # 13+ is usually fine; 17 is safe & recent
)

print(f"Saved ONNX model to {onnx_path}")


C:\Users\Huize\AppData\Local\Temp\ipykernel_30880\1595373125.py:33: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1123 21:38:50.409000 30880 site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `LogitsWrapper([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `LogitsWrapper([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 51 of general pattern rewrite rules.
Saved ONNX model to ships_vit_tiny.onnx


## wrapped together .onnx

In [22]:
import torch
import torch.nn as nn

# assume `model` is your fine-tuned ViTForImageClassification
model.eval()
model.to("cpu")

class LogitsWrapper(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model

    def forward(self, pixel_values):
        # pixel_values: (B, 3, 224, 224)
        outputs = self.base_model(pixel_values=pixel_values)
        return outputs.logits  # (B, 2)

wrapped_model = LogitsWrapper(model).to("cpu")
wrapped_model.eval()

dummy_input = torch.randn(1, 3, 224, 224)  # same shape as pixel_values

onnx_path = "ships_vit_tiny_sing.onnx"

torch.onnx.export(
    wrapped_model,
    dummy_input,
    onnx_path,
    input_names=["pixel_values"],
    output_names=["logits"],
    dynamic_axes={
        "pixel_values": {0: "batch_size"},
        "logits": {0: "batch_size"},
    },
    opset_version=17,   # or 13–17, whatever your torch supports
)

print(f"Saved ONNX model to {onnx_path}")


C:\Users\Huize\AppData\Local\Temp\ipykernel_30880\104917450.py:25: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1123 21:45:43.628000 30880 site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `LogitsWrapper([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `LogitsWrapper([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 51 of general pattern rewrite rules.
Saved ONNX model to ships_vit_tiny_sing.onnx


In [18]:
!pip install onnxruntime

In [ ]:
import onnxruntime as ort
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

onnx_path = "ships_vit_tiny_sing.onnx"

# 1. Create ONNX Runtime session
# If you have GPU support in ORT, you can add "CUDAExecutionProvider" first.
session = ort.InferenceSession(
    onnx_path,
    providers=["CPUExecutionProvider"]
)

all_labels = []
all_preds = []

# 2. Loop over your PyTorch test_loader
model_input_name = session.get_inputs()[0].name  # should be "pixel_values"
# when building test_loader for ONNX:
test_loader_onnx = make_dataloader(
    x_test_path, y_test_path,
    image_processor,
    batch_size=1,
    shuffle=False,
)

for pixel_values, labels in test_loader_onnx:
    # pixel_values: torch.Tensor (B, 3, 224, 224)
    # labels: torch.Tensor (B,)

    # Move to CPU & convert to numpy
    pixel_values_np = pixel_values.detach().cpu().numpy().astype(np.float32)

    # 3. Run ONNX inference
    (logits,) = session.run(
        ["logits"],
        {model_input_name: pixel_values_np},
    )  # logits: (B, 2)

    preds = np.argmax(logits, axis=-1)  # (B,)

    all_labels.extend(labels.cpu().numpy().tolist())
    all_preds.extend(preds.tolist())

# 4. Compute metrics
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

acc = accuracy_score(all_labels, all_preds)

# military = 1 is the positive class
precision, recall, f1, _ = precision_recall_fscore_support(
    all_labels,
    all_preds,
    average="binary",
    pos_label=1,
)

print(f"ONNX Test Accuracy          : {acc:.4f}")
print(f"ONNX Precision (military=1) : {precision:.4f}")
print(f"ONNX Recall (military=1)    : {recall:.4f}")
print(f"ONNX F1 (military=1)        : {f1:.4f}")


ONNX Test Accuracy          : 0.9829
ONNX Precision (military=1) : 0.9913
ONNX Recall (military=1)    : 0.9744
ONNX F1 (military=1)        : 0.9828


: 

## Evaluate OOD - on fire

In [4]:
import os
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import onnxruntime as ort
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# ----------------------------
# Config
# ----------------------------
MODEL_PATH = "models/ships_vit_tiny_sing.onnx"        # <-- change to your actual .onnx filename
OOD_DIR    = "dataset/ood_on_fire"           # folder with civilian_*.jpg / military_*.jpg

IMAGE_SIZE = (224, 224)                      # as requested
BATCH_SIZE = 1                               # safer for ONNX (avoids shape issues)


# ----------------------------
# Dataset
# ----------------------------

class OODShipsDataset(Dataset):
    """
    Folder structure:
        dataset/ood_on_fire/
            civilian_....jpg / png / ...
            military_....jpg / png / ...
    Labels:
        civilian_* -> 0
        military_* -> 1
    """
    def __init__(self, root, transform=None):
        super().__init__()
        self.root = root
        self.transform = transform
        self.samples = []

        exts = {".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp"}

        for fname in os.listdir(root):
            fpath = os.path.join(root, fname)
            if not os.path.isfile(fpath):
                continue

            ext = os.path.splitext(fname)[1].lower()
            if ext not in exts:
                continue

            lower = fname.lower()
            if lower.startswith("civilian"):
                label = 0
            elif lower.startswith("military"):
                label = 1
            else:
                # skip files that don't match the naming convention
                continue

            self.samples.append((fpath, label))

        if not self.samples:
            raise RuntimeError(f"No valid images found in {root}")

        print(f"Found {len(self.samples)} OOD images in {root}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label


# ----------------------------
# Build DataLoader
# ----------------------------

transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),  # (150, 150)
    transforms.ToTensor(),          # (3, H, W), float32 in [0,1]
])

ood_dataset = OODShipsDataset(OOD_DIR, transform=transform)

ood_loader = DataLoader(
    ood_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)


# ----------------------------
# Load ONNX model
# ----------------------------

# Create ONNX Runtime session
session = ort.InferenceSession(
    MODEL_PATH,
    providers=["CPUExecutionProvider"],   # or add CUDAExecutionProvider if you have ORT GPU
)

# Get input/output names
input_name  = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
print(f"ONNX model loaded from {MODEL_PATH}")
print(f"Input name: {input_name}, Output name: {output_name}")


# ----------------------------
# Inference + metrics
# ----------------------------

all_labels = []
all_preds  = []

for imgs, labs in ood_loader:
    # imgs: (B, 3, 150, 150) torch.float32
    # labs: (B,) integers (0 or 1)

    # Move to numpy for ONNX Runtime
    imgs_np = imgs.numpy().astype(np.float32)  # shape: (B, 3, 150, 150)

    # Run ONNX inference
    logits = session.run([output_name], {input_name: imgs_np})[0]  # (B, num_classes)

    preds = np.argmax(logits, axis=-1)  # (B,)

    all_labels.extend(labs.numpy().tolist())
    all_preds.extend(preds.tolist())

y_true = np.array(all_labels)
y_pred = np.array(all_preds)

acc = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true,
    y_pred,
    average="binary",
    pos_label=1,  # military = 1
)

print(f"\nOOD ONNX evaluation on {len(y_true)} images:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision (military=1): {precision:.4f}")
print(f"Recall    (military=1): {recall:.4f}")
print(f"F1        (military=1): {f1:.4f}")


Found 61 OOD images in dataset/ood_on_fire
ONNX model loaded from models/ships_vit_tiny_sing.onnx
Input name: pixel_values, Output name: logits

OOD ONNX evaluation on 61 images:
Accuracy : 0.8525
Precision (military=1): 0.7895
Recall    (military=1): 0.9677
F1        (military=1): 0.8696
